In [2]:
import kagglehub
kagglehub.login()
data = kagglehub.competition_download('deep-learning-spring-2025-project-2')
print('Data source import complete.')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
from tensorflow.keras import layers
import os
import shutil
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("/content/kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 600)
!kaggle competitions download -c deep-learning-spring-2025-project-2
!unzip deep-learning-spring-2025-project-2.zip -d /content/

100%|██████████| 2.06M/2.06M [00:00<00:00, 4.18MB/s]

Extracting files...


Data source import complete.
Archive:  deep-learning-spring-2025-project-2.zip
  inflating: /content/test_unlabelled.pkl  
Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
!pip3 install transformers datasets evaluate accelerate peft bitsandbytes -q
!pip3 install nvidia-ml-py3 -q # For GPU monitoring, optional
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import os
import pandas as pd
import torch
import numpy as np
from transformers import (
    RobertaForSequenceClassification,
    RobertaTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, PeftModel, TaskType # Added TaskType
from datasets import load_dataset, Dataset, ClassLabel
import evaluate # Use Hugging Face evaluate
import pickle
# Note: bitsandbytes is imported but used via transformers' from_pretrained arguments

# --- Configuration ---
BASE_MODEL = 'roberta-base'
DATASET_NAME = 'ag_news'
OUTPUT_DIR = "roberta-agnews-lora-results" # More descriptive name
MAX_PARAMETER_LIMIT = 1000000

# --- Load Dataset and Tokenizer ---
print(f"Loading dataset: {DATASET_NAME}")
# Load the full training split first
dataset = load_dataset(DATASET_NAME, split='train')
# Load the actual test split for final unlabelled prediction (if needed later)
# test_unlabelled_split = load_dataset(DATASET_NAME, split='test')

print(f"Loading tokenizer for: {BASE_MODEL}")
tokenizer = RobertaTokenizer.from_pretrained(BASE_MODEL)

# --- Preprocessing ---
def preprocess(examples):
    # Truncation needed, padding will be handled by DataCollator
    tokenized = tokenizer(examples['text'], truncation=True, padding=False, max_length=256) # Adjust max_length if needed
    return tokenized

print("Tokenizing dataset...")
# Apply tokenization (remove original text column)
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])
# Rename label column
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch") # Set format early

# --- Dataset Info and Split ---
num_labels = tokenized_dataset.features['labels'].num_classes
class_names = tokenized_dataset.features["labels"].names
print(f"Number of labels: {num_labels}")
print(f"Label names: {class_names}")
id2label = {i: label for i, label in enumerate(class_names)}
label2id = {label: i for i, label in id2label.items()} # Useful for model config

import numpy as np
token_lengths = [len(x) for x in tokenized_dataset['input_ids']]
lower_percentile = 1 # Example: remove shortest 1%
upper_percentile = 99 # Example: remove longest 1%
lower_thresh = np.percentile(token_lengths, lower_percentile)
upper_thresh = np.percentile(token_lengths, upper_percentile)
print(f"Filtering training data: Keeping lengths between {lower_thresh:.0f} and {upper_thresh:.0f} tokens.")

original_size = len(tokenized_dataset)
# Ensure you are filtering the dataset object correctly
indices_to_keep = [i for i, length in enumerate(token_lengths) if lower_thresh <= length <= upper_thresh]
filtered_dataset = tokenized_dataset.select(indices_to_keep)
new_size = len(filtered_dataset)
print(f"Filtered dataset size: {new_size} (removed {original_size - new_size} samples)")

# NOW split the filtered_dataset
split_datasets = filtered_dataset.train_test_split(test_size=0.1, seed=456)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']
# ... continue with training ...

# Split the tokenized training set into train and validation
# Using a slightly larger validation set might be more robust
#split_datasets = tokenized_dataset.train_test_split(test_size=0.1, seed=42) # Example: 10% for validation
#train_dataset = split_datasets['train']
#eval_dataset = split_datasets['test']
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(eval_dataset)}")

# --- Data Collator ---
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# --- Load Base Model ---
print(f"Loading base model: {BASE_MODEL}")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",          # Recommended type: Normalized Float 4
    bnb_4bit_compute_dtype=torch.float16 # Use bfloat16 for computation if supported, else torch.float16
    # bnb_4bit_use_double_quant=True,    # Optional: Can sometimes improve results slightly
)

model = RobertaForSequenceClassification.from_pretrained(
    BASE_MODEL,
    num_labels=num_labels,
    id2label=id2label,
    #label2id=label2id,
    #quantization_config=bnb_config, # Pass the configuration
    # device_map="auto" # Often recommended with quantization for memory balancing across GPUs if >1
)

# --- Sanity Check: Parameters BEFORE LoRA ---
total_params_before = sum(p.numel() for p in model.parameters())
trainable_params_before = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n--- Parameters BEFORE LoRA ---")
print(f"Total parameters:     {total_params_before:,}")
print(f"Trainable parameters: {trainable_params_before:,}")
print("-" * 30)


# --- LoRA Configuration ---
# *** CRITICAL TUNING AREA ***
# Adjust 'r' and 'target_modules' to meet the <= 1M trainable parameter constraint
# while maximizing performance. Start higher and reduce 'r' if needed.
peft_config = LoraConfig(
    r=7,                   # Rank (Start higher, e.g., 8, 16, 32, and decrease if over 1M params)
    lora_alpha=77,          # Scaling factor (often r*2, but tunable)
    lora_dropout=0.01,       # Dropout probability for LoRA layers (0.05 or 0.1 is common)
    bias='none',            # Usually 'none' for LoRA
    target_modules=[        # Modules to apply LoRA to (check model architecture)
        "query",
        "value",
         "key",            # Optional: Adding key increases params
        #"dense"           # Optional: Add fully connected layers in attention
    ],
    task_type=TaskType.SEQ_CLS, # Specify task type
)

# --- Apply LoRA ---
print("\nApplying LoRA configuration...")
peft_model = get_peft_model(model, peft_config)

# --- CRITICAL CHECK: Trainable Parameters AFTER LoRA ---
peft_model.print_trainable_parameters()
trainable_params_after = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
print(f"Calculated Trainable parameters: {trainable_params_after:,}")

if trainable_params_after > MAX_PARAMETER_LIMIT:
    print(f"\n🚨 WARNING: Trainable parameters ({trainable_params_after:,}) exceed the limit ({MAX_PARAMETER_LIMIT:,})!")
    print("🚨 CONSIDER REDUCING 'r' or removing modules from 'target_modules'.")
    # exit() # Uncomment to stop if the constraint is violated
else:
    print(f"\n✅ Trainable parameters ({trainable_params_after:,}) are within the limit ({MAX_PARAMETER_LIMIT:,}).")
print("-" * 30)

Loading dataset: ag_news
Loading tokenizer for: roberta-base
Tokenizing dataset...
Number of labels: 4
Label names: ['World', 'Sports', 'Business', 'Sci/Tech']
Filtering training data: Keeping lengths between 26 and 123 tokens.
Filtered dataset size: 117675 (removed 2325 samples)
Train dataset size: 105907
Validation dataset size: 11768
Loading base model: roberta-base


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- Parameters BEFORE LoRA ---
Total parameters:     124,648,708
Trainable parameters: 124,648,708
------------------------------

Applying LoRA configuration...
trainable params: 980,740 || all params: 125,629,448 || trainable%: 0.7807
Calculated Trainable parameters: 980,740

✅ Trainable parameters (980,740) are within the limit (1,000,000).
------------------------------


In [9]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Ensure predictions are numpy arrays
    if isinstance(predictions, tuple): # Sometimes Trainer passes logits and other outputs
        predictions = predictions[0]
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    preds = np.argmax(predictions, axis=1)
    results = accuracy_metric.compute(predictions=preds, references=labels)
    return results # Return dictionary like {'accuracy': 0.95}

# --- Training Arguments ---
# *** HYPERPARAMETER TUNING AREA ***
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    # --- Key args for accuracy/overfitting ---
    learning_rate=3e-5,           # Keep for now, but tunable (3e-5, 8e-5, 1e-4)
    num_train_epochs=6,           # TUNED: Reduced epochs (was 6)
    optim="adamw_torch",          # CONFIRMED: Use AdamW
    weight_decay=0.1,            # CONFIRMED: Standard weight decay
    # --- Evaluation / Saving Best ---
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,  # CRITICAL: Use best model found during validation
    metric_for_best_model="loss", # Monitor the evaluation loss
    greater_is_better=False,   # CRITICAL
    # --- Other Args ---
    logging_strategy="steps",
    logging_steps=100,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    fp16=torch.cuda.is_available(), # Use GPU boost
    #seed=42,
    report_to="none",
    lr_scheduler_type='cosine',
    gradient_checkpointing_kwargs={'use_reentrant': True}
    # ... other args if needed ...
)

# --- Trainer Setup ---
def get_trainer(model):
    return Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,         # Use the validation split for evaluation during training
        tokenizer=tokenizer,               # Pass tokenizer for padding/logging
        data_collator=data_collator,
    )

peft_lora_finetuning_trainer = get_trainer(peft_model)


<ipython-input-9-2b09f226e9eb>:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
# --- Training ---
print("\n--- Starting LoRA Fine-Tuning ---")
train_result = peft_lora_finetuning_trainer.train()
print("--- Training Finished ---")

# Log training metrics
metrics = train_result.metrics
peft_lora_finetuning_trainer.log_metrics("train", metrics)
peft_lora_finetuning_trainer.save_metrics("train", metrics)
peft_lora_finetuning_trainer.save_state() # Saves trainer state

# --- Evaluation on Validation Set ---
print("\n--- Evaluating Best Model on Validation Set ---")
eval_metrics = peft_lora_finetuning_trainer.evaluate(eval_dataset=eval_dataset) # Use the same eval_dataset
peft_lora_finetuning_trainer.log_metrics("eval", eval_metrics)
peft_lora_finetuning_trainer.save_metrics("eval", eval_metrics)
print(eval_metrics)

# --- Save Final Model (Best Adapter) ---
# The trainer saves the full checkpoint, but we often only need the adapter
best_adapter_path = os.path.join(OUTPUT_DIR, "best_adapter")
peft_model.save_pretrained(best_adapter_path)
print(f"Best LoRA adapter saved to: {best_adapter_path}")

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",          # Recommended type: Normalized Float 4
#     bnb_4bit_compute_dtype=torch.float16 # Use bfloat16 for computation if supported, else torch.float16
#     # bnb_4bit_use_double_quant=True,    # Optional: Can sometimes improve results slightly
# )

# quantized_model = RobertaForSequenceClassification.from_pretrained(
#     BASE_MODEL,
#     num_labels=num_labels,
#     id2label=id2label,
#     label2id=label2id,
#     quantization_config=bnb_config,
# )

# quantized_model = PeftModel.from_pretrained(quantized_model, "trained_peft_model")




--- Starting LoRA Fine-Tuning ---


Step,Training Loss,Validation Loss,Accuracy
100,1.212700,0.519391,0.872026
200,0.369000,0.302718,0.899048
300,0.323800,0.281137,0.904742
400,0.280800,0.271282,0.910350
500,0.275700,0.262852,0.910945
600,0.260500,0.257747,0.914939
700,0.259300,0.253694,0.916298
800,0.256900,0.250432,0.916893
900,0.264400,0.246753,0.917828
1000,0.266600,0.238440,0.919273


--- Training Finished ---
***** train metrics *****
  epoch                    =        6.0
  total_flos               = 32562158GF
  train_loss               =     0.2107
  train_runtime            = 1:12:12.08
  train_samples_per_second =    146.683
  train_steps_per_second   =      2.292

--- Evaluating Best Model on Validation Set ---


***** eval metrics *****
  epoch                   =        6.0
  eval_accuracy           =     0.9388
  eval_loss               =     0.1783
  eval_runtime            = 0:00:18.28
  eval_samples_per_second =    643.531
  eval_steps_per_second   =     10.062
{'eval_loss': 0.17827866971492767, 'eval_accuracy': 0.938817131203263, 'eval_runtime': 18.2866, 'eval_samples_per_second': 643.531, 'eval_steps_per_second': 10.062, 'epoch': 6.0}
Best LoRA adapter saved to: roberta-agnews-lora-results/best_adapter


In [11]:
from torch.utils.data import DataLoader
import tqdm

In [12]:
unlabelled_pickle_path = "test_unlabelled.pkl"
unlabelled_df = pd.read_pickle(unlabelled_pickle_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_dataset = Dataset.from_dict({"text": unlabelled_df["text"]})

# Tokenize function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=256)

# Apply tokenizer
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Create PyTorch DataLoader for batching
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=64, collate_fn=data_collator)

# Prediction loop
#quantized_model.eval()
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()if isinstance(v, torch.Tensor)}
        model_inputs = {k: v for k, v in batch.items() if k in tokenizer.model_input_names + ["attention_mask"]}
        outputs = peft_model(**model_inputs)
        preds = torch.argmax(outputs.logits, dim=-1)
        all_predictions.extend(preds.cpu().numpy())

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [13]:
df = pd.DataFrame({
    "ID": list(range(len(all_predictions))),   # ID ✅
    "label": all_predictions
})
df.to_csv("submission.csv", index=False)
print("✅ Batched predictions complete. Saved to submission.csv.")

✅ Batched predictions complete. Saved to submission.csv.
